In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import transforms, datasets
import torch.optim as optim
import torch.nn.functional as tfunc
from torch.utils.data import Dataset
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau
from PIL import Image
import torch.nn.functional as func
from util_datasets import GaussianNoise, UniformNoise
import csv

from sklearn.metrics.ranking import roc_auc_score
import sklearn.metrics as metrics
import random

%load_ext autoreload

%autoreload 2

In [2]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(123)
device = torch.device("cuda" if use_cuda else "cpu")


class_names = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 
               'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 
               'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']

imgtransResize = (320, 320)
imgtransCrop = 224

In [3]:
use_cuda

True

In [4]:
from trainer import CheXpertTrainer 
from chexpertClass import CheXpertData
from denseNet121 import DenseNet121
from utils import *
from ood_evaluate import *

In [5]:
#TRANSFORM DATA SEQUENCE
normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
transformList = []
#transformList.append(transforms.Resize(imgtransCrop))
transformList.append(transforms.RandomResizedCrop(imgtransCrop))
transformList.append(transforms.RandomHorizontalFlip())
transformList.append(transforms.ToTensor())
transformList.append(normalize)      
transformSequence=transforms.Compose(transformList)

In [6]:
#CheXpert dataset loading
chex_datasetValid = CheXpertData('datasets/chexpert-small/CheXpert-v1.0-small/valid.csv',transformSequence, preload = True, policy="ones")
chex_datasetTrain = CheXpertData('datasets/chexpert-small/CheXpert-v1.0-small/train.csv',transformSequence, policy="ones")
print("Train set: ", len(chex_datasetTrain))
print("Valid set: ", len(chex_datasetValid))
datasetValid, datasetTrain = random_split(chex_datasetTrain, [766, len(chex_datasetTrain) - 766])
chex_test, chex_train = random_split(datasetTrain, [1000, len(datasetTrain) - 1000])

#split datasets into train,valid,test
chex_valid = torch.utils.data.ConcatDataset([chex_datasetValid, datasetValid])
print("New train set: ", len(chex_train))
print("New valid set: ", len(chex_valid))
print("New test set: ", len(chex_test))
dataLoaderTrain = DataLoader(dataset=chex_train, batch_size=32, shuffle=True,  num_workers=2, pin_memory=True)
dataLoaderValid = DataLoader(dataset=chex_valid, batch_size=32, shuffle=False,  num_workers=1, pin_memory=True)
dataLoaderTest = DataLoader(dataset=chex_test, batch_size=32, shuffle=False,  num_workers=1, pin_memory=True)

Train set:  223414
Valid set:  234
New train set:  221648
New valid set:  1000
New test set:  1000


In [7]:
#NIH dataset loading
nih_dataset = datasets.ImageFolder(root='datasets/nih-small/small', transform = transformSequence)
nih_test, nih_train = random_split(nih_dataset, [1000, len(nih_dataset) - 1000])
print("NIH train set: ", len(nih_train))
print("NIH valid set: ", len(nih_test))
dataLoaderNIH = DataLoader(dataset=nih_test, batch_size=32, shuffle=False,  num_workers=0, pin_memory=True)

NIH train set:  4606
NIH valid set:  1000


In [ ]:
model = DenseNet121(len(class_names)).to(device)

In [ ]:
model = torch.nn.DataParallel(model).to(device)

In [ ]:
checkpoint_path = 'cheXpert_github/model_ones_3epoch_densenet.tar'

In [ ]:
trainer = CheXpertTrainer(model, class_names, checkpoint_path, use_cuda, device, epoch = 3)
dataLoaders = [dataLoaderTrain, dataLoaderValid]
lost_train, loss_eval = trainer.train(dataLoaders)

In [ ]:
#ind_scores, ind_gt, ind_conf = evaluate_ood(dataLoaderTest, 'confidence', model, device)

In [ ]:
#experiment without the pretrained weights
trainer = CheXpertTrainer(model = model, class_names = class_names, use_cuda = use_cuda, device = device, epoch = 3)
dataLoaders = [dataLoaderTrain, dataLoaderValid]
new_lost_train, new_loss_eval = trainer.train(dataLoaders)

### New Traininig for Model with adjusted DenseNet121

In [9]:
from denseNet121_v2 import *
from trainer_v2 import *

model = DenseNet121v2(len(class_names)).to(device)
model = torch.nn.DataParallel(model).to(device)
trainer = CheXpertTrainerv2(model = model, class_names = class_names, use_cuda = use_cuda, device = device, epoch = 3)
dataLoaders = [dataLoaderTrain, dataLoaderValid]
new_lost_train, new_loss_eval = trainer.train(dataLoaders)

  0%|          | 0/32 [00:00<?, ?it/s]

Train Loss after epoch  1 :  0.28676509857177734
Eval Loss after epoch  1 :  0.317018061876297
AUROC scores after epoch  1 :
Validation set: 


Test Progress..: 100%|██████████| 32/32 [00:14<00:00,  2.23it/s]


AUROC mean  0.7453734910648304
No Finding   0.8589944134078213
Enlarged Cardiomediastinum   0.5861370378409666
Cardiomegaly   0.7988642976102094
Lung Opacity   0.7290660225442835
Lung Lesion   0.7250520833333333
Edema   0.8441668155113903
Consolidation   0.668569816491577
Pneumonia   0.6851068853021979
Atelectasis   0.6696706552706552
Pneumothorax   0.7602727305697602
Pleural Effusion   0.8477728344879151
Pleural Other   0.777326565143824
Fracture   0.6412435233160622
Support Devices   0.8429851940776311


  0%|          | 0/32 [00:00<?, ?it/s]

...

AUROC mean  0.7507619531747965
No Finding   0.8437882415536047
Enlarged Cardiomediastinum   0.5566825186968447
Cardiomegaly   0.7552287122686625
Lung Opacity   0.7084983896940418
Lung Lesion   0.7038802083333333
Edema   0.8468798123108263
Consolidation   0.6640761439049947
Pneumonia   0.7181490384615383
Atelectasis   0.688888888888889
Pneumothorax   0.7920671187997921
Pleural Effusion   0.8433610515575494
Pleural Other   0.7910659898477157
Fracture   0.7286454478164324
Support Devices   0.8694557823129251


  0%|          | 0/32 [00:00<?, ?it/s]

Train Loss after epoch  3 :  0.35181933641433716
Eval Loss after epoch  3 :  0.3312612473964691
AUROC scores after epoch  3 :
Validation set: 


Test Progress..: 100%|██████████| 32/32 [00:14<00:00,  2.22it/s]


AUROC mean  0.7519502087208524
No Finding   0.8517052407555199
Enlarged Cardiomediastinum   0.6100137471962955
Cardiomegaly   0.7978008307081085
Lung Opacity   0.7236151368760064
Lung Lesion   0.7275260416666667
Edema   0.8419510405453027
Consolidation   0.6608788521991457
Pneumonia   0.7046381353021979
Atelectasis   0.7042279202279202
Pneumothorax   0.7843300813597844
Pleural Effusion   0.8519615497420263
Pleural Other   0.6941455160744501
Fracture   0.6983863804589194
Support Devices   0.8761224489795918
